<a href="https://colab.research.google.com/github/AlejoBI/IO/blob/main/TallerParametricas_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
#Intalamos la libreria para hacer peticiones a Fixer.io para cambio peso/dolar
!pip install requests

#Instalamos la librería / software de optimización de Google
!pip install ortools

#@title ⚙️ Ejecutar esta celda para elegir el solucionador de PLEyM

#Importar el módulo de PL de G-OR
from ortools.linear_solver import pywraplp
import numpy as np
import requests

#Elegir el solucionador
solucionador =  'SCIP' #@param ["GLOP", "PDLP", "SCIP"]
solver = pywraplp.Solver.CreateSolver(solucionador)

In [66]:
# Función para calcular distancia euclidiana en km
def distancia_euclidiana(coord1, coord2):
    R = 6371  # Radio de la Tierra en kilómetros
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distancia = R * c
    return distancia

# Función para calcular distancia Manhattan en km
def distancia_manhattan_km(coord1, coord2):
    # Aproximación: 1 grado de latitud o longitud ~ 111 km
    km_por_grado = 111.0
    dlat = np.abs(coord1[0] - coord2[0]) * km_por_grado
    dlon = np.abs(coord1[1] - coord2[1]) * km_por_grado
    distancia = dlat + dlon
    return distancia

# Función para calcular el costo de combustible
def calcular_costo_combustible(distancia, autonomia_por_galon=100, precio_por_litro=5200):
    litros_por_galon = 3.78541
    distancia *= 2  # Ida y vuelta
    galones_necesarios = distancia / autonomia_por_galon
    litros_necesarios = galones_necesarios * litros_por_galon
    costo_total = litros_necesarios * precio_por_litro
    return costo_total

# Conversión de COP a USD usando fixer.io
def convertir_cop_a_usd_fixer(monto):
    api_key = '68f155f6cea70124ee7187c30c8da5d8'
    url = f"http://data.fixer.io/api/latest?access_key={api_key}&symbols=USD,COP"
    try:
        response = requests.get(url)
        data = response.json()
        if data['success']:
            tasa_usd = data['rates']['USD']
            tasa_cop = data['rates']['COP']
            tasa_cop_a_usd = tasa_usd / tasa_cop
            monto *= 10  # Según lo especificado en el problema
            conversion = monto * tasa_cop_a_usd
            return conversion
        else:
            raise ValueError("Error en la API de Fixer")
    except requests.exceptions.RequestException as e:
        raise RuntimeError(f"Error de conexión: {e}")

# Coordenadas de las cocinas
coordenadas_cocinas = np.array([
    [3.448400976866912, -76.5325112913085],  # Cocina 1
    [3.444784779336403, -76.50872895216395], # Cocina 2
    [3.4849005479200437, -76.50897881657532] # Cocina 3
])

# Coordenadas de los colegios
coordenadas_colegios = np.array([
    [3.416223218777475, -76.55103032681261],  # Colegio 1
    [3.436751684248191, -76.52538732928453],  # Colegio 2
    [3.441777634328216, -76.53579089623783],  # Colegio 3
    [3.4823585560497063, -76.50229272170012], # Colegio 4
    [3.4189628289329654, -76.48625892561115]  # Colegio 5
])

# Capacidad máxima mensual de las cocinas en paquetes
capacidad_cocinas = [200, 240, 180]  # Cocina 1, 2, 3

# Demanda máxima mensual de los colegios en paquetes
demanda_colegios = [60, 50, 130, 80, 40]  # Colegio 1, 2, 3, 4, 5

# Costos fijos por cocina en USD
costos_fijos = [1200, 1300, 1700]  # Cocina 1, 2, 3

# Crear solver SCIP
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variables de decisión: x[cocina, colegio] = paquetes de desayuno desde la cocina al colegio
x = {}
for i in range(len(coordenadas_cocinas)):
    for j in range(len(coordenadas_colegios)):
        x[i, j] = solver.IntVar(0, solver.infinity(), f"x_{i}_{j}")

# Variables binarias para seleccionar cocinas
y = {}
for i in range(len(coordenadas_cocinas)):
    y[i] = solver.BoolVar(f"y_{i}")

# Restricción: Solo se pueden usar exactamente 2 cocinas
solver.Add(sum(y[i] for i in range(len(coordenadas_cocinas))) == 2)

# Restricciones de capacidad de las cocinas
for i in range(len(coordenadas_cocinas)):
    solver.Add(sum(x[i, j] for j in range(len(coordenadas_colegios))) <= capacidad_cocinas[i] * y[i])

# Restricciones de demanda de los colegios
for j in range(len(coordenadas_colegios)):
    solver.Add(sum(x[i, j] for i in range(len(coordenadas_cocinas))) >= demanda_colegios[j])

# Función objetivo: maximizar beneficio
margen_contribucion = 62.5  # USD por paquete

# Cálculo del total de contribución
total_beneficio = solver.Sum(
    margen_contribucion * x[i, j] for i in range(len(coordenadas_cocinas)) for j in range(len(coordenadas_colegios))
)

# Cálculo de costos fijos
total_costos_fijos = solver.Sum(costos_fijos[i] * y[i] for i in range(len(coordenadas_cocinas)))

# Cálculo de costos de transporte (10 veces el costo de combustible)
total_costos_combustible = solver.Sum(
    10 * calcular_costo_combustible(distancia_euclidiana(coordenadas_cocinas[i], coordenadas_colegios[j])) * x[i, j]
    for i in range(len(coordenadas_cocinas)) for j in range(len(coordenadas_colegios))
)

# Maximizar el beneficio neto
solver.Maximize(total_beneficio - total_costos_fijos - total_costos_combustible)

# Ejecutar el solver
status = solver.Solve()

# Resultados
if status == pywraplp.Solver.OPTIMAL:
    print('Solución óptima encontrada:')
    for i in range(len(coordenadas_cocinas)):
        for j in range(len(coordenadas_colegios)):
            if x[i, j].solution_value() > 0:
                print(f'Paquetes de Cocina {i+1} a Colegio {j+1}: {x[i, j].solution_value()}')
    print(f'Beneficio total: {solver.Objective().Value()} USD')
else:
    print('No se encontró una solución óptima.')


Solución óptima encontrada:
Paquetes de Cocina 1 a Colegio 1: 60.0
Paquetes de Cocina 1 a Colegio 2: 10.0
Paquetes de Cocina 1 a Colegio 3: 130.0
Paquetes de Cocina 2 a Colegio 2: 40.0
Paquetes de Cocina 2 a Colegio 4: 80.0
Paquetes de Cocina 2 a Colegio 5: 40.0
Beneficio total: -3692108.1933763726 USD


In [52]:
############################ PRUEBAS ##############################

# Función para calcular distancia euclidiana en km
def distancia_euclidiana(coord1, coord2):
    R = 6371  # Radio de la Tierra en kilómetros
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distancia = R * c
    return distancia

# Función para calcular distancia Manhattan en km
def distancia_manhattan_km(coord1, coord2):
    # Aproximación: 1 grado de latitud o longitud ~ 111 km
    km_por_grado = 111.0
    dlat = np.abs(coord1[0] - coord2[0]) * km_por_grado
    dlon = np.abs(coord1[1] - coord2[1]) * km_por_grado
    distancia = dlat + dlon
    return distancia

# Función para calcular la distancia según el método elegido
def calcular_distancia(cocina_idx, colegio_idx, metodo):
    cocina = cocinas[cocina_idx - 1]
    colegio = colegios[colegio_idx - 1]

    if metodo == 1:
        res = distancia_euclidiana(cocina, colegio)
        print(f"\nLa distancia entre la Cocina {cocina_idx} y el Colegio {colegio_idx} usando el método {'euclidiana' if metodo == 1 else 'manhattan'} es: {res:.2f} km")
        return res
    elif metodo == 2:
        res = distancia_manhattan_km(cocina, colegio)
        print(f"\nLa distancia entre la Cocina {cocina_idx} y el Colegio {colegio_idx} usando el método {'euclidiana' if metodo == 1 else 'manhattan'} es: {res:.2f} km")
        return res
    else:
        raise ValueError("Método no válido. Elija 1 para 'euclidiana' o 2 para 'manhattan'.")

# Función para seleccionar la cocina, colegio y el método
def seleccionar_opciones():
    print("Seleccione una cocina:")
    for i in range(len(cocinas)):
        print(f"{i+1}: Cocina {i+1}")
    cocina_idx = int(input("Ingrese el número de la cocina: "))

    print("\nSeleccione un colegio:")
    for j in range(len(colegios)):
        print(f"{j+1}: Colegio {j+1}")
    colegio_idx = int(input("Ingrese el número del colegio: "))

    print("\nSeleccione el método de distancia:")
    print("1: Euclidiana")
    print("2: Manhattan")
    metodo = int(input("Ingrese el número del método: "))

    return cocina_idx, colegio_idx, metodo


# ------------------------- COMBUSTIBLE ----------------------------

def calcular_costo_combustible(distancia, autonomia_por_galon=100, precio_por_litro=5200):
    # Constante: 1 galón equivale a 3.78541 litros
    litros_por_galon = 3.78541

    # Distancia * 2 para trayecto de ida y vuelta
    distancia *= 2

    # 1. Calcular cuántos galones se necesitan para recorrer la distancia
    galones_necesarios = distancia / autonomia_por_galon

    # 2. Convertir galones a litros
    litros_necesarios = galones_necesarios * litros_por_galon

    # 3. Calcular el costo total
    costo_total = litros_necesarios * precio_por_litro

    print(f"Para recorrer {distancia:.3f} km, necesitas gastar {costo_total:.3f} COP en combustible.")
    # Retornar el mensaje con el costo calculado
    return f"{costo_total:.3f}"


# ------------------------ COP A USD ----------------------------

def convertir_cop_a_usd_fixer(monto):
    api_key = '68f155f6cea70124ee7187c30c8da5d8'
    url = f"http://data.fixer.io/api/latest?access_key={api_key}&symbols=USD,COP"

    try:
        response = requests.get(url)
        data = response.json()

        if data['success']:
            # Obtener la tasa de COP a USD
            tasa_usd = data['rates']['USD']
            tasa_cop = data['rates']['COP']
            tasa_cop_a_usd = tasa_usd / tasa_cop

            # 10 veces el monto del combustible segun lo acordado
            monto *= 10

            conversion = monto * tasa_cop_a_usd

            print(f"Tasa actual COP a USD: {tasa_cop_a_usd}")
            print(f"El monto {monto} COP equivale a {conversion:.3f} USD")
            return f"{conversion:.3f}"
        else:
          print("Error al obtener los datos:", data['error']['info'])
          return

    except requests.exceptions.RequestException as e:
        print(f"Error de conexión o solicitud: {e}")
        return


# --------------------------- EJECUCION DEL PROGRAMA ----------------------------

# Coordenadas de las cocinas
cocinas = np.array([
    [3.448400976866912, -76.5325112913085],  # Cocina 1
    [3.444784779336403, -76.50872895216395], # Cocina 2
    [3.4849005479200437, -76.50897881657532] # Cocina 3
])

# Coordenadas de los colegios
colegios = np.array([
    [3.416223218777475, -76.55103032681261],  # Colegio A
    [3.436751684248191, -76.52538732928453],  # Colegio B
    [3.441777634328216, -76.53579089623783],  # Colegio C
    [3.4823585560497063, -76.50229272170012], # Colegio D
    [3.4189628289329654, -76.48625892561115]  # Colegio E
])

band = 1
while(band <= 2):
  print(f"\n------ PETICION #{band} ------")
  band += 1

  #Seleccionar opciones
  cocina_idx, colegio_idx, metodo = seleccionar_opciones()

  #Distancias
  distancia = calcular_distancia(cocina_idx, colegio_idx, metodo)

  #Combustible
  costo_combustible = float(calcular_costo_combustible(distancia))

  #COP to USD
  costo_combustible_usd = float(convertir_cop_a_usd_fixer(costo_combustible))


------ PETICION #1 ------
Seleccione una cocina:
1: Cocina 1
2: Cocina 2
3: Cocina 3
Ingrese el número de la cocina: 1

Seleccione un colegio:
1: Colegio 1
2: Colegio 2
3: Colegio 3
4: Colegio 4
5: Colegio 5
Ingrese el número del colegio: 1

Seleccione el método de distancia:
1: Euclidiana
2: Manhattan
Ingrese el número del método: 1

La distancia entre la Cocina 1 y el Colegio 1 usando el método euclidiana es: 4.13 km
Para recorrer 8.253 km, necesitas gastar 1624.498 COP en combustible.
Tasa actual COP a USD: 0.0002359685542292223
El monto 16244.98 COP equivale a 3.833 USD

------ PETICION #2 ------
Seleccione una cocina:
1: Cocina 1
2: Cocina 2
3: Cocina 3
Ingrese el número de la cocina: 2

Seleccione un colegio:
1: Colegio 1
2: Colegio 2
3: Colegio 3
4: Colegio 4
5: Colegio 5
Ingrese el número del colegio: 2

Seleccione el método de distancia:
1: Euclidiana
2: Manhattan
Ingrese el número del método: 2

La distancia entre la Cocina 2 y el Colegio 2 usando el método manhattan es: 2.7